In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd
import numpy as np
import conda_scripts
import forward_run
import matplotlib.pyplot as plt
import conda_scripts.plot_help as ph
from conda_scripts import sv_budget
import matplotlib as mpl
import numpy as np
import flopy
import geopandas as gpd

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
main = 'postprocess'
fold =  'pestaws_v1'

out_folder = os.path.join(main, fold)
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok =True)

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)

In [ ]:
workspace

In [ ]:
import flopy.modflow.mfhob as hfb
import flopy.modflow.mfhyd as hyd

In [ ]:
df = pd.read_csv(r"C:\GSP\waterlevel\GIS\hydro_experiment\hydros__v3_SON_allmodmonths_to_20240808\hydros_predicted_SON.csv")

In [ ]:
df = df.query('`Station Name`.str.contains("_mod") == False', engine='python')

In [ ]:
df = df.drop_duplicates(['Station Name', 'datetime'])

In [ ]:
df.loc[:,'Depth'] = "Other"
df.loc[df.Shallow,'Depth'] = "Shallow"
df.loc[df.Deep,'Depth'] = "Deep"

df = df.loc[~(df.Depth=='Other')]
df.drop_duplicates('Station Name').Depth.value_counts()



In [ ]:
lrc = df.drop_duplicates('Station Name').set_index('Station Name').apply( lambda row: ml.modelgrid.intersect(row['Easting'], row['Northing']),axis = 1)

lrc = pd.DataFrame(lrc.to_list(), index = lrc.index, columns=['i','j'])

In [ ]:
df = pd.merge(df, lrc, left_on = 'Station Name', right_index = True)

In [ ]:
df.query("Deep==True | Shallow==True").columns

In [ ]:
df

In [ ]:
dates =conda_scripts.sv_budget.sv_modflow.get_dates()
dates.loc[:,'date'] = dates.date-  pd.offsets.MonthBegin(1)
dates.head()
dates = dates.astype({'date':'datetime64[ns]'})
print(dates.dtypes)

In [ ]:
print(df.shape)
df = pd.merge(df.astype({'datetime':'datetime64[ns]'}), dates, left_on = 'datetime', right_on = 'date')
print(df.shape)

In [ ]:
stats = conda_scripts.wiski.wiski.get_stations()
stats = stats.loc[:,['station_no', 'station_name']].rename(columns = {'station_name':"Station Name"})

In [ ]:
stats

In [ ]:
df = pd.merge(df, stats, on = "Station Name")

In [ ]:
df

In [ ]:
# add standard deviation to obs

# export to csv for other users

In [ ]:

df.set_index(['Station Name','date','kstp','Depth','i','j',"Easting",'Northing']).loc[:,['predicted']].to_csv(os.path.join('..','waterlevel', 'GWLE','gwle_estimates_for_hydobs.csv'), float_format='%.3f')



# [flopy link to hydmod](https://flopy.readthedocs.io/en/latest/source/flopy.modflow.mfhyd.html)

In [ ]:
zone = ml.bas6.ibound.array[0]
zone2 = ml.bas6.ibound.array[1]
zone = zone+zone2
plt.imshow(zone)
# plt.figure()
# plt.imshow(zone2)
zone = conda_scripts.arich_functions.array2rc(zone,name = 'zone').drop(columns = ['row', 'column'])
# zone2 = conda_scripts.arich_functions.array2rc(zone2,name = 'zone2').drop(columns = ['row', 'column'])

In [ ]:
obsdata = df.copy()
obsdata = obsdata.drop_duplicates("Station Name")
obsdata = pd.merge(obsdata, zone, on = ['i','j'])
print(obsdata.shape)
# obsdata = pd.merge(obsdata, zone2, on = ['i','j'])
# print(obsdata.shape)
obsdata.loc[:,'xl'] ,obsdata.loc[:,'yl'] = ml.modelgrid.get_local_coords(obsdata.Easting.values, obsdata.Northing.values)
obsdata.loc[:,['pckg','arr','intyp', 'klay']] = ['BAS','HD',"C",-111]
obsdata.loc[obsdata.Shallow,'klay'] = 0
obsdata.loc[obsdata.Deep,'klay'] = 3
#set layer to layer 1 where zone/ibound ==1 (ie highlands)
obsdata.loc[obsdata.zone==1,'klay'] = 0
# remove locations no in ibound2
# obsdata = obsdata.loc[obsdata.zone2!=0,:]
obsdata = obsdata.loc[obsdata.klay>=0]
obsdata = obsdata.loc[:,['pckg','arr','intyp', 'klay', 'xl', 'yl','Station Name']]
print(obsdata.shape)

In [ ]:
obsdata.klay.value_counts()

In [ ]:
obsdata

In [ ]:
hyd.ModflowHyd.get_default_dtype()

In [ ]:
ml.remove_package('HYD')

In [ ]:
ml.model_ws

In [ ]:
hello = hyd.ModflowHyd(ml, 
           nhyd=obsdata.shape[0], 
           ihydun=None, 
           hydnoh=-9999.0, 
           # obsdata=[['BAS', 'HD', 'C', 0, 0.0, 0.0, 'HOBS1']], 
                          obsdata=obsdata.values, 
           extension=['hyd', 'hyd.bin'], 
           unitnumber=70, 
           filenames="hyd.hyd")
hello.write_file()

# load hydmod output

In [ ]:
obsdata.drop_duplicates('Station Name').to_csv(os.path.join('..','waterlevel', 'GWLE','hydobs_locations.csv'), float_format='%.3f')

In [ ]:
import forward_run
from importlib import reload
reload(forward_run)

In [ ]:
forward_run.run_all_hyd_obs(workspace)

In [ ]:
s = forward_run.load_hydobs(workspace)
s = forward_run.create_diff_for_hydobs(s,numper_diff=5)
s = forward_run.down_sample_hydobs(s, numper_diff=5, keep_every=4)
s = forward_run.create_obs_from_hyd(s)


# creating 10-year rolling average from GWLE

In [ ]:
bigobj = forward_run.load_hydobs(workspace)
abs_small = forward_run.down_sample_hydobs(bigobj, numper_diff = 5,keep_every = 4)
abs_obs = forward_run.create_obs_from_hyd(abs_small)


# rolling observations
roll = forward_run.rolling_mean(bigobj,nyears=10)
roll_obs = forward_run.create_obs_from_hyd(roll)
roll_obs


In [ ]:
s.loc[:,'year'] = s.date.dt.year
s.loc[:,'month'] = s.date.dt.month
f = s.set_index('year').loc[:,['month','Son0001']].set_index('month', append = True).unstack()
f = f.where(f.isnull(),1)
f = f.droplevel(0,1).fillna(0)

In [ ]:
import numpy as np 
from pandas import DataFrame
import seaborn as sns
%matplotlib inline

# Index= ['aaa', 'bbb', 'ccc', 'ddd', 'eee']
# Cols = ['A', 'B', 'C', 'D']
# df = DataFrame(abs(np.random.randn(5, 4)), index=Index, columns=Cols)

sns.heatmap(f, annot=True)

In [ ]:
def load_hydobs(workspace):
    out = flopy.utils.HydmodObs(os.path.join(workspace, 'output', 'SV_hyd.hyd'),)
    out = out.get_dataframe(start_datetime='12/1/1969')
    out = out.rename(columns = lambda x: x[6:]  if len(x)>12 else x)
    out = out.drop(columns = 'totim')

    out = out.reindex(pd.date_range('1975-01-01', freq = 'MS', periods = 525))

    return out

def create_obs_from_hyd(sim):
    sim = sim.stack()
    sim.index = sim.index.set_names(['Station', 'date'])
    sim = sim.swaplevel().to_frame('meas').reset_index()

    return sim




# load hydmod output

In [ ]:
out = flopy.utils.HydmodObs(os.path.join(workspace, 'output', 'SV_hyd.hyd'),)
out = out.get_dataframe(start_datetime='12/1/1969')
out = out.rename(columns = lambda x: x[6:]  if len(x)>12 else x)
out = out.drop(columns = 'totim')
display(out.head())
display(out.tail())

In [ ]:
import importlib

In [ ]:
importlib.reload(forward_run)

# reshape and shuffle the GWLE data to look like hydmod dataframe

In [ ]:
res = df.loc[:,['Station Name', "predicted",'date']].set_index(['date','Station Name']).unstack(1)
res.columns = res.columns.droplevel(0)
res = res.rename_axis(None, axis=1)
res = res.rename_axis(None, axis=0)
diff_mat = out - res.reindex_like(out)

diff_mat = diff_mat.dropna(how = 'all', axis = 0)

diff_mat = diff_mat.loc[:,diff_mat.abs().max().sort_values(ascending = True).index]

# export the GWLE data in format to ingest via pest

In [ ]:
gwle_diff = forward_run.create_diff_for_hydobs(res,numper_diff=5)
gwle_diff = forward_run.down_sample_hydobs(gwle_diff, numper_diff=5, keep_every=4)
gwle_diff = forward_run.create_obs_from_hyd(gwle_diff)

In [ ]:
# gwle_abs = forward_run.create_diff_for_hydobs(res,numper_diff=5)
gwle_abs = forward_run.down_sample_hydobs(res, numper_diff=5, keep_every=4)
gwle_abs = forward_run.create_obs_from_hyd(gwle_abs)

In [ ]:
#rolling 18month
roll_short = forward_run.rolling_mean(res,nyears = None, nmonths=18)
roll_short_obs = forward_run.create_obs_from_hyd(roll_short)
roll_short_obs.tail(3)

In [ ]:
# rolling observations
roll = forward_run.rolling_mean(res,nyears=8, nmonths = None )
roll_obs = forward_run.create_obs_from_hyd(roll)
roll_obs

In [ ]:

forward_run.rolling_mean(res, nyears = 8, )


In [ ]:
forward_run.rolling_mean(res, nmonths = 18,nyears = None).tail()


In [ ]:
# pd.date_range(ml.dis.start_datetime,periods = ml.dis.nper, freq = 'M')

In [ ]:
print(f'there are {gwle_diff.shape[0]} observations in the GWLE observation drawdown elev file')
print(f'there are {gwle_abs.shape[0]} observations in the GWLE absolute elev file')
print(f'there are {roll_obs.shape[0]} observations in the GWLE observation rolling elev file')
print(f'there are {roll_short_obs.shape[0]} observations in the GWLE observation rolling 18month elev file\n')


f_abs = os.path.join(os.path.join(workspace, "GWLE_OBS", 'gwle_asbolute_observed_heads.csv'))
print(f"writing absolute heads to {f_abs}")
f_diff = os.path.join(os.path.join(workspace, "GWLE_OBS", 'gwle_drawdown_observed_heads.csv'))
print(f"writing drawdon heads to {f_diff}")
f_roll = os.path.join(os.path.join(workspace, "GWLE_OBS", 'gwle_rolling_observed_heads.csv'))
print(f"writing rolling heads to {f_roll}")
f_roll_short = os.path.join(os.path.join(workspace, "GWLE_OBS", 'gwle_rolling_18month_observed_heads.csv'))
print(f"writing rolling heads to {f_roll_short}")

gwle_abs.to_csv(f_abs)
gwle_diff.to_csv(f_diff)
roll_obs.to_csv(f_roll)
roll_short_obs.to_csv(f_roll_short)

f_abs = os.path.join('..','waterlevel', 'GWLE','gwle_asbolute_observed_heads.csv')
print(f"writing absolute heads to {f_abs}")
f_diff = os.path.join('..','waterlevel', 'GWLE', 'gwle_drawdown_observed_heads.csv')
print(f"writing drawdon heads to {f_diff}")
f_roll = os.path.join('..','waterlevel', 'GWLE', 'gwle_rolling_observed_heads.csv')
print(f"writing f_roll heads to {f_roll}")

f_roll_short = os.path.join(os.path.join('..', 'waterlevel',"GWLE", 'gwle_rolling_18month_observed_heads.csv'))
print(f"writing rolling heads to {f_roll_short}")

gwle_abs.to_csv(f_abs)
gwle_diff.to_csv(f_diff)
roll_obs.to_csv(f_roll)
roll_short_obs.to_csv(f_roll_short)

In [ ]:
outf = os.path.join('..','waterlevel','hydros')
os.makedirs(outf,exist_ok=True)

In [ ]:
import matplotlib.dates as mdates

for n,stat in gwle_abs.groupby('Station'):
    ax = stat.plot(x = 'date', y='meas', label = 'GWLE',marker = '.')
    roll_obs.query(f"Station=='{n}'").plot(x = 'date', y='meas', ax =ax, marker = 's' , mfc = 'w', label = 'GWLE - 10-Year')
    roll_short_obs.query(f"Station=='{n}'").plot(x = 'date', y='meas', ax =ax, marker = 'o', mfc = 'k', label = 'GWLE - 18-Month ')
    
    # Set major ticks to every 5 years starting at 1970
    # ax.xaxis.set_major_locator(mdates.YearLocator(5))  # Every 5 years
    # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  # Format ticks as years
    ax.set_title(n); ax.grid(True)
    # Set x-axis limit to ensure it starts from 1970
    plt.xlim(pd.Timestamp('1970-01-01'), pd.Timestamp('2020-01-01'))
    # Rotate and align the tick labels
    plt.xticks(rotation=45, ha='right')
    conda_scripts.plot_help.yrange_(ax)
    plt.savefig(os.path.join(outf, n+'.png'), dpi = 250, bbox_inches = "tight")
    plt.close()
    

# export again the station info

In [ ]:
import cartopy.crs as ccrs
import contextily as ctx

In [ ]:
z = np.genfromtxt(os.path.join(ml.model_ws, 'model_arrays', 'zonation_3.csv'), delimiter = ' ')

zotther = z.copy()
zotther[zotther>8] = 0

zones = {i:zotther  for i in range(7) }
zones[1] = z
plt.imshow(zones[1])

z = conda_scripts.arich_functions.array2rc(zones[1],'zone').astype({'zone':int})

aliases = {1: 'Bay', 2: 'EastSide', 3: 'SouthCent', 4: 'Kenwood', 5: 'VOM', 6: 'AguaCal',7:'WestSide',8:'CitySon',9:'Highlands'}
z.loc[:,'zone'] =z.loc[:,'zone'].replace(aliases)
z = z.query("zone!=0")

geoms = conda_scripts.arich_functions.get_model_shp(ml.modelgrid, 2226).drop(columns = ['row','col'])

z = gpd.GeoDataFrame(pd.merge(z, geoms, on = ['i','j']))
zplot = z.dissolve('zone').reset_index()

In [ ]:

plt.figure(figsize = (6,8), dpi = 250)
ax = plt.subplot(1,1,1, projection = ccrs.epsg(2226))
zplot.exterior.plot(ax = ax,  )
zplot.plot('zone', ax = ax, alpha =.5  )
conda_scripts.plot_help.label_poly(zplot,ax, 'zone',color = 'g', text_color='k') 
conda_scripts.arich_functions.add_basemaps(ax)
ax.set_title('zone names')
plt.savefig('map_of_zones_for_PEST.png', dpi = 250)

In [ ]:
vcounts = pd.read_csv(r"C:\GSP\waterlevel\GIS\hydro_experiment\hydros__v3_SON_allmodmonths_to_20240808\seasinfo_w_predicted_SON.csv")
vcounts = vcounts.loc[:,'Station Name'].value_counts().to_frame("num_meas")
vcounts = vcounts.loc[~vcounts.index.str.contains('mod')]

In [ ]:
vcounts

In [ ]:
important_hydros_folder

In [ ]:
dfinfo = pd.merge(df.loc[:,['Station Name', 'station_no', 
       'Site', 'Easting', 'Northing', 'Latitude',
       'Longitude',  'rasterelevation', 'slope',
       'Simple_Bou', 'Complete_B', 'isostatic',  'Geol_Krig',
 
      'Depth', 'i', 'j', 
       ]].drop_duplicates('Station Name'),
                  z, on = ['i','j'], how = 'left')
dfinfo = dfinfo.drop(columns = ['geometry','row','column'])

dfinfo = pd.merge(dfinfo, vcounts,on = 'Station Name' )

dfinfo.to_csv(os.path.join(workspace, "GWLE_OBS",'gwle_station_info.csv'), float_format='%.3f')


dfinfo.to_csv(os.path.join('..','waterlevel', 'GWLE','gwle_station_info.csv'), float_format='%.3f')

In [ ]:
df.loc[df.loc[:,'Station Name'].isin(diff_mat.loc[:,~diff_mat.columns.isin(diff_mat.dropna(how = 'all', axis = 1).columns)].columns)]

In [ ]:
print('all of these columns are probably null in the GWLE values')
diff_mat.loc[:,~diff_mat.columns.isin(diff_mat.dropna(how = 'all', axis = 1).columns)]

In [ ]:
from importlib import reload
import helpers
from helpers import make_plot
reload(helpers)

In [ ]:
from helpers import make_plot

In [ ]:
c = 0
for   stat in d:
    # print(stat)
    row = df.query(f"`Station Name`=='{stat}'")
    
    rowind = row['i'].unique()[0]
    column = row['j'].unique()[0]
    
    x = row['Easting'].unique()[0]
    y = row['Northing'].unique()[0]
    fig, ax = helpers.make_plot('',x,y)

    ax.set_title(stat)
    res.loc[:,stat].rename("GWLE").plot(ax = ax, label = "GWLE")
    out.loc[:,stat].plot(ax = ax ,label = "OWHM")
    out.loc[:,stat].head(1).plot(ax = ax ,c = 'r', marker = 'o', label = "Starting")
    ax.legend()
    
    # lays = row.iloc[[0],:].reset_index().at[0,'mlays']
    # lays = ', '.join([str(j+1) for j in lays])
    # ax.set_title( "".join(row.obsname.unique()[0].split("_")[:-1])  )

    ph.yrange_(ax)
    ax.grid(True)
    print(stat, rowind, column, row.shape,os.path.abspath(os.path.join(out_folder,'hydros', stat+'.png' )))
    plt.savefig(os.path.join(out_folder,'hydros', stat+'.png' ))
    plt.close(fig)
    
    

In [ ]:
out_folder

In [ ]:
import pandas as pd

def process_csv(input_file, output_file, threshold_x, threshold_y, threshold_z, threshold_a, threshold_b):
    """
    Processes a CSV file based on given criteria.
    
    Args:
    input_file: Path to the input CSV file.
    output_file: Path for the output CSV file.
    threshold_x: Threshold value for station number SL1 0329.
    threshold_y: Threshold value for station number so10330.
    threshold_z: Threshold value for station number son 0346.
    threshold_a: Threshold value for station number s o n 0549.
    threshold_b: Threshold value for station number s o n 0550.
    """
    
    # Read the CSV file into a Pandas DataFrame
    df = pd.read_csv(input_file)
    print(df.shape)
    # Remove rows with specific station numbers
    df = df[~df['station_name'].str.contains('Son0067')]
    df = df[~df['station_name'].str.contains('Son0079')]
    df = df[~df['station_name'].str.contains('Son0085')]
    df = df[~df['station_name'].str.contains('Son0346')]
    df = df[~df['station_name'].str.contains('Son0082')]
    df = df[~df['station_name'].str.contains('Son0097')]
    df = df[~df['station_name'].str.contains('Son0130')]
    df = df[~df['station_name'].str.contains('Son0148')]
    df = df[~df['station_name'].str.contains('Son0216')]
    df = df[~df['station_name'].str.contains('Son0220')]
    df = df[~df['station_name'].str.contains('Son0221')]
    df = df[~df['station_name'].str.contains('Son0223')]
    df = df[~df['station_name'].str.contains('Son0231')]
    df = df[~df['station_name'].str.contains('Son0233')]
    df = df[~df['station_name'].str.contains('Son0247')]
    
    # Replace values based on station numbers and thresholds
    df.loc[(df['station_name'] == 'Son0329') & (df['Manual Measurement'] > threshold_x), 'Manual Measurement'] = np.nan
    df.loc[(df['station_name'] == 'Son0330') & (df['Manual Measurement'] < threshold_y), 'Manual Measurement'] = np.nan

    df.loc[(df['station_name'] == 'Son0346') & (df['Manual Measurement'] > threshold_z), 'Manual Measurement'] = np.nan
    df.loc[(df['station_name'] == 'Son0549') & (df['Manual Measurement'] < threshold_a), 'Manual Measurement'] = np.nan
    df.loc[(df['station_name'] == 'Son0550') & (df['Manual Measurement'] < threshold_b), 'Manual Measurement'] = np.nan
    
    df.loc[(df['station_name'] == 'Son0040') & (df['Manual Measurement'] > 975), 'Manual Measurement'] = np.nan
    df.loc[(df['station_name'] == 'Son0185') & (df['Manual Measurement'] > 820), 'Manual Measurement'] = np.nan
    
    # Convert timestamp to datetime
    t = pd.to_datetime(df['Timestamp'], utc=True)
    
    # Filter data
    # Filter data
    df = df[~((df['station_name'] == 'Son0330') & (
        (t >= pd.to_datetime('2015-01-01', utc=True)) & (df['Manual Measurement'] >= -35)))]


    
    df = df.dropna(subset = 'Manual Measurement')
    
    print(df.shape)


    df.to_csv(output_file, index=False)

    return df
    
# Example usage:
input_file = 'your_input_file.csv'
output_file = 'filtered_data.csv'
threshold_x = -2
threshold_y = -55
threshold_z = 80
threshold_a = 60
threshold_b = 60

input_file = r"C:\GSP\waterlevel\regression_data\all_gw_for_surf_2024_05_01.csv"
output_file = r"C:\GSP\waterlevel\regression_data\all_gw_for_surf_2024_08_08.csv"
dfal = process_csv(input_file, output_file, threshold_x, threshold_y, threshold_z, threshold_a, threshold_b)

In [ ]:
dfal = conda_scripts.utils.load_all_gw_wiski.load_all_gw(download=False,filter_manual=True,outfolder=r"C:\GSP\waterlevel\regression_data", outfile="all_gw_for_surf_2024_08_08.csv")

In [ ]:
    for stat in ['Son0085','Son0329',
'Son0330',
'Son0346',
'Son0549',
'Son0550']:
        fig, ax = plt.subplots()
        cdf= dfal.query(f"station_name=='{stat}'")
        for t, ddd in cdf.groupby('Param_reclass'):

            ddd.plot.scatter(ax = ax, x = 'Timestamp', y = 'Manual Measurement', label = t, legend = True )
        ax.set_title(stat)

In [ ]:
# Replace values based on station numbers and thresholds
dfal.loc[(dfal['station_name'] == 'Son0329') ]
# df.loc[(df['station_name'] == 'Son0330') & (df['Manual Measurement'] < threshold_y), 'Manual Measurement'] = np.nan
# df.loc[(df['station_name'] == 'Son0346') & (df['Manual Measurement'] > threshold_z), 'Manual Measurement'] = np.nan
# df.loc[(df['station_name'] == 'Son0549') & (df['Manual Measurement'] < threshold_a), 'Manual Measurement'] = np.nan
# df.loc[(df['station_name'] == 'Son0550') & (df['Manual Measurement'] < threshold_b), 'Manual Measurement'] = np.nan

In [ ]:

bad = out.loc[:,out.nunique()==1]
bad = list(map(lambda x: x[6:], bad))
bad

In [ ]:
df.loc[:,'measname'] = df.loc[:,'Station Name']+"_"+df.loc[:,'kstp'].astype(str)

In [ ]:
df.query(f"`Station Name`=='Son0001'").sort_values('kstp').kstp.duplicated().sum()

In [ ]:
f= df.query(f"`Station Name`=='Son0001'").sort_values('kstp')
f.loc[f.kstp.duplicated()]

In [ ]:
import random
import string

def generate_random_string(length=10):
    # Define the pool of characters (letters and digits)
    characters = string.ascii_letters + string.digits
    # Randomly select 'length' characters from the pool
    random_string = ''.join(random.choice(characters) for _ in range(length))
    return random_string

In [ ]:
def dd(ml, station, hds):

    q = hds.query(f"`Station Name`=='{station}'").sort_values('kstp')
    q = q.drop_duplicates('kstp')
    # print(q)
    hall = []
    
    for i in range(q.shape[0]-1):
        ts = q.iloc[[i,i+1],:].loc[:,['kstp','predicted']].values
        wellname = q.iloc[[i],:].loc[:,['measname']].values[0][0]
        obsname = q.iloc[[i+1],:].loc[:,['measname']].values[0][0]+'_dd0'
        names =  copy.deepcopy([q.iloc[[i],:].loc[:,['measname']].values[0][0]+'_hd1', 
                                q.iloc[[i+1],:].loc[:,['measname']].values[0][0]+'xxdd0'])
        # names = [generate_random_string(length=10),generate_random_string(length=10)]
        obsname = copy.deepcopy(wellname+'_hd0')
        row = q.i.unique()[0]
        column = q.j.unique()[0]
        # print(wellname)
        print(obsname)
        print(names)
        h = hfb.HeadObservation(ml, 
                    tomulth=1.0, 
                    obsname= q.iloc[[i+1],:].loc[:,['measname']].values[0][0]+'xxhd0', 
                    layer=0, 
                    row=row, 
                    column=column, 
                    irefsp=-i, 
                    roff=0.0, 
                    coff=0.0,
                    itt=2,
                    tmax=None, 
                    mlay=None, 
                    time_series_data=ts, 
                    names=names)

        hall.extend([copy.deepcopy(h)])
        
    return hall

heads = dd(ml, 'Son0001', df)
hob = flopy.modflow.ModflowHob(ml, iuhobsv=51, hobdry=-9999.,

                               obs_data=heads)
hob.write_file()

In [ ]:
hob.obs_data[0].time_series_data

In [ ]:
len('SV-F06-01_62')

In [ ]:
h = hfb.HeadObservation(ml, 
                    tomulth=1.0, 
                    obsname='HOBS', 
                    layer=0, 
                    row=0, 
                    column=0, 
                    irefsp=None, 
                    roff=0.0, 
                    coff=0.0,
                    itt=1,
                    tmax=None, 
                    mlay=None, 
                    time_series_data=None, 
                    names=None)

In [ ]:
df

In [ ]:
ml.modelgrid.get_local_coords(obsdata.Easting, obsdata.Northing)

In [ ]:
flopy.utils.HydmodObs(os.path.join(workspace, 'output', 'SV_hyd.hyd'))

In [ ]:
os.listdir(os.path.join(workspace, 'output'))